## Get Best-Performing Hyper Parameters

In [94]:
MONGO_COLLECTION = "CB_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "CB_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

# MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "SC_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

MONGO_COLLECTION

'CB_RE-RANKER_HYPER_PARAM_VD'

In [95]:
assert MONGO_COLLECTION.endswith("VD")

In [96]:
cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type"]
if "INSENS" in MONGO_COLLECTION:
    cols = [c for c in cols if c != "loss_type"]
# cols

In [97]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [98]:
df = pd.DataFrame(results)
# df.tail(3)[["asof"] + cols]

In [99]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
37,0.746014,0.777294,0.717153,2019-06-18 01:55:37.752000,2,0.0025,300,1,0.01,ml,1
57,0.745665,0.789796,0.706204,2019-06-18 02:38:17.475000,1,0.0100,300,1,0.01,ml,1
35,0.745506,0.774067,0.718978,2019-06-18 01:51:39.666000,2,0.0005,300,1,0.01,ml,2
53,0.745016,0.776415,0.716058,2019-06-18 02:30:03.563000,1,0.0025,300,1,0.01,ml,1
33,0.744965,0.769141,0.722263,2019-06-18 01:47:36.756000,2,0.0005,300,1,0.01,ml,1


In [100]:
# Best ones that are not 2
# df[df.best_top_n != 2][cols].head()